# Romance

In [1]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "romance_translated"
method = "pre-translate"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_gpt_response(claim, model=model):
    try:
        user_prompt = prompt_template.format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_gpt_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


/Users/aryan/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: False
Wrong response
Model Output: Final Answer: True
Final Answer: True, Actual Answer: True
Correct response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Mostly true
Correct response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: True
Wrong response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: True
Wrong response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Half true
Wrong response
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: True
Wrong response
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: Half true
Correct response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Mostly true
Correct response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actu

# Indo-Aryan

In [2]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "indo_aryan_translated"
method = "pre-translate"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_gpt_response(claim, model=model):
    try:
        user_prompt = prompt_template.format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_gpt_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: Final Answer: False
Final Answer: False, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: False
Correct response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: False
Correct response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: Half true
Wrong response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: True
Final Answer: True, Actual Answer: Half true
Wrong response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: True
Final Answer: True, Actual Answer: Mostly false
Wrong response
Model Output: Final 

# Kartvelian

In [3]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "kartvelian_translated"
method = "pre-translate"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_gpt_response(claim, model=model):
    try:
        user_prompt = prompt_template.format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_gpt_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: False
Correct response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: True
Wrong response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: True
Wrong response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: Mostly true
Wrong response
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: Half true
Correct response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: Half true
Wrong response
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: False
Wrong re

# Slavic

In [4]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "slavic_translated"
method = "pre-translate"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_gpt_response(claim, model=model):
    try:
        user_prompt = prompt_template.format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_gpt_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: False
Wrong response
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: True
Wrong response
Model Output: Final Answer: Mostly False
Final Answer: Mostly false, Actual Answer: False
Wrong response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: False
Correct response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: False
Correct response
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: Half True
Final Answer: Half true, Actual Answer: Mostly true
Wrong response
Model Output: Final Answer: Mostly False
Final Answer: Mostly false, Actual Answer: Mostly false
Correct response
Model Output: Final Answer: Mostly False
Final Answer: Mostly false, Actu

# Turkic

In [5]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "turkic_translated"
method = "pre-translate"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_gpt_response(claim, model=model):
    try:
        user_prompt = prompt_template.format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_gpt_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: Final Answer: False
Final Answer: False, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: Mostly False
Final Answer: Mostly false, Actual Answer: Mostly false
Correct response
Model Output: Final Answer: Mostly False
Final Answer: Mostly false, Actual Answer: Mostly false
Correct response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Half true
Wrong response
Model Output: Final Answer: Mostly True
Final Answer: Mostly true, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: Mostly true
Wrong response
Model Output: Final Answer: True
Final Answer: True, Actual Answer: Half true
Wrong response
Model Output: Final Answer: Mostly False
Final Answer: Mostly false, Actual Answer: Mostly false
Correct response
Model Output: Final Answer: True
Final Answer: True, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: Half True
Final Answer: Half 